# Task 2: Recommendation Engine - Skeleton Notebook

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the Used Cars dataset; there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* Please consider this notebook as an example and not to set specific requirements. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [1]:
import pandas as pd

## Load the Data

For this example, we use a simplified version of the dataset with only 100 data sample, each with only 6 features

In [2]:
df_sample = pd.read_csv('data/sg-used-cars-final-simplified.csv')

df_sample.head()

,listing_id,make,power,engine_cap,mileage,price
0,1006025,bmw,135.0,1997.0,NaN,82300
1,994672,land rover,202.0,2993.0,25843.0,427900
2,921142,honda,95.6,1496.0,2000.0,109800
3,1008328,bmw,185.0,1998.0,57386.0,166600
4,1010661,NaN,96.0,1498.0,76000.0,59400


## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [4]:
def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = None
    
    # Extract all **kwargs input parameters
    # and set the used parameters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value
    #####################################################
    ## Compute your recommendations
    #
    # This is where your magic happens. Of course, you can call methods
    # defined in this notebook or in external Python (.py) scripts
    #
        
    # Here, we just return the input row k times
    # Ideally, you recommendations will be much better
    df_result = pd.DataFrame([row] * k, index=None)
        
    # Return the dataset with the k recommendations
    return df_result

## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [9]:
# Pick a row id of choice
#row_id = 10
row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

row = df_sample.iloc[row_id]
pd.DataFrame([row])

,listing_id,make,power,engine_cap,mileage,price
20,1022692,mitsubishi,57.0,1193.0,NaN,46100


## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [10]:
k = 3

df_recommendations = get_top_recommendations(row, k=k)

df_recommendations.head(k)

,listing_id,make,power,engine_cap,mileage,price
20,1022692,mitsubishi,57.0,1193.0,NaN,46100
20,1022692,mitsubishi,57.0,1193.0,NaN,46100
20,1022692,mitsubishi,57.0,1193.0,NaN,46100
